In [7]:
from Utils.my_utils import *
from Recommenders.KNN.ItemKNNCFRecommender import *
import numpy as np
from Evaluation.Evaluator import EvaluatorHoldout
from HyperparameterTuning.run_hyperparameter_search import runHyperparameterSearch_Collaborative
from Recommenders.NonPersonalizedRecommender import *
import os

In [8]:
np.random.seed(1234)
users, items, users_to_eval = readData()
URM, user_originalID_to_index, item_originalID_to_index, item_index_to_originalID = preProcess()
URM_train_full, URM_train, URM_test, URM_validation = splitURM(URM)

In [9]:
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 2602 (20.6%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 2227 (17.6%) Users that have less than 1 test interactions


In [10]:
output_folder_path = "result_experiments/"
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

saved_folder_path = "result_best_models/"
if not os.path.exists(saved_folder_path):
    os.makedirs(saved_folder_path)
    
recommender_class = ItemKNNCFRecommender    
n_cases = 10
n_random_starts = int(n_cases*0.3)
metric_to_optimize = "MAP"   
cutoff_to_optimize = 10

In [11]:
runHyperparameterSearch_Collaborative(recommender_class,
       URM_train = URM_train,
       URM_train_last_test = URM_train_full,
       metric_to_optimize = metric_to_optimize,
       cutoff_to_optimize = cutoff_to_optimize,
       evaluator_validation_earlystopping = evaluator_validation,
       evaluator_validation = evaluator_validation,
       evaluator_test = evaluator_test,
       output_folder_path = output_folder_path,
       parallelizeKNN = True,
       allow_weighting = True,
       resume_from_saved = True,
       save_model = "best",
       similarity_type_list = ['cosine', 'jaccard', "asymmetric", "dice", "tversky"],
       n_cases = n_cases,
       n_random_starts = n_random_starts)

On recommender <class 'Recommenders.KNN.ItemKNNCFRecommender.ItemKNNCFRecommender'> Exception SearchBayesianSkopt: Resuming 'ItemKNNCFRecommender_jaccard' Failed due to inconsistent data. There cannot be both a valid result and an exception for the same case.


multiprocessing.pool.RemoteTraceback: 
"""
Traceback (most recent call last):
  File "c:\Users\Simone\anaconda3\lib\multiprocessing\pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
  File "c:\Users\Simone\anaconda3\lib\multiprocessing\pool.py", line 48, in mapstar
    return list(map(*args))
  File "c:\Users\Simone\Desktop\RecommenderSystemsChallenge\HyperparameterTuning\run_hyperparameter_search.py", line 454, in run_KNNRecommender_on_similarity_type
    hyperparameterSearch.search(recommender_input_args,
  File "c:\Users\Simone\Desktop\RecommenderSystemsChallenge\HyperparameterTuning\SearchBayesianSkopt.py", line 329, in search
    hyperparameters_list_input, result_on_validation_list_saved = self._resume_from_saved()
  File "c:\Users\Simone\Desktop\RecommenderSystemsChallenge\HyperparameterTuning\SearchBayesianSkopt.py", line 188, in _resume_from_saved
    assert is_exception == is_validation_valid,\
AssertionError: SearchBayesianSkopt: Resuming 'ItemKNNCFRecom

In [12]:
# load the model with all best parameters
recommender_object = ItemKNNCFRecommender(URM_train_full)
recommender_object.load_model(output_folder_path, file_name = recommender_object.RECOMMENDER_NAME + "_tversky_best_model_last.zip" )

ItemKNNCFRecommender: URM Detected 213 ( 1.7%) users with no interactions.
ItemKNNCFRecommender: URM Detected 102 ( 0.5%) items with no interactions.
ItemKNNCFRecommender: Loading model from file 'result_experiments/ItemKNNCFRecommender_tversky_best_model_last.zip'


ItemKNNCFRecommender: Loading complete


In [13]:
# load the TopPop model with all best parameters
recommender_object_top_pop = TopPop(URM_train_full)
recommender_object_top_pop.load_model(saved_folder_path, file_name = recommender_object_top_pop.RECOMMENDER_NAME + "_my_own_save.zip" )

TopPopRecommender: URM Detected 213 ( 1.7%) users with no interactions.
TopPopRecommender: URM Detected 102 ( 0.5%) items with no interactions.
TopPopRecommender: Loading model from file 'result_best_models/TopPopRecommender_my_own_save.zip'
TopPopRecommender: Loading complete


In [14]:
# recommend items to users
rec_items_ind = []
for user_id in users_to_eval:
    if(user_id in users):
        # recommend users for which we already have some ratings
        rec_items_ind.append(recommender_object.recommend(user_originalID_to_index[user_id], cutoff=10))
    else:
        # recommend users with no stored rating
        rec_items_ind.append(recommender_object_top_pop.recommend(user_id, cutoff=10, remove_seen_flag=False))

In [15]:
# evaluate recommender
evaluate_algorithm(URM_test, recommender_object)

MAP = 0.0771


In [16]:
storeRecommendations(users_to_eval, rec_items_ind, item_index_to_originalID)

In [ ]:
# save the model with all best parameters overwriting the old one
#recommender_object.save_model(saved_folder_path, file_name = recommender_object.RECOMMENDER_NAME + "_my_own_save.zip" )